# Introduction
This file will be used to format the training set for the different tests

We will begin by clearing folders that have been created by this same script. So we can get a fresh setup

In [ ]:
import os
# !pip install -U ultralytics
import ultralytics
# from ultralytics import settings
# # Get the current working directory
# datasets_folder = os.path.join(os.getcwd(), "datasets")
# # Update the datasets_dir setting with the actual path value
# if not os.path.exists(datasets_folder):
#     os.makedirs(datasets_folder)
# settings.update({"datasets_dir": datasets_folder})
!yolo checks

In [ ]:
%reset -f
import os
import shutil

# Path to the 'datasets' folder
dataset_folder = "datasets"

# List of folders to keep (Modify this if needed)
folders_to_keep = []  # Example: ["images", "labels"]

# Check if the dataset folder exists
if not os.path.exists(dataset_folder):
    print(f"Error: The folder '{dataset_folder}' does not exist.")
else:
    # Iterate through all items in the dataset folder
    for item in os.listdir(dataset_folder):
        item_path = os.path.join(dataset_folder, item)
        
        # Check if the item is not in the keep list
        if item not in folders_to_keep:
            # Remove the folder or file
            if os.path.isdir(item_path):
                shutil.rmtree(item_path)  # Remove directories
                print(f"Removed folder: {item_path}")
            else:
                os.remove(item_path)  # Remove files
                print(f"Removed file: {item_path}")
print("Cleanup completed.")




The following code downloads the COCO dataset from github.

In [ ]:
from ultralytics.utils.downloads import download
from pathlib import Path

# Define the dataset folder path
dataset_dir = Path().parent / "datasets"  # Set dataset path relative to the script

# Ensure the dataset folder exists
dataset_dir.mkdir(parents=True, exist_ok=True)

# Download labels
segments = True  # segment or box labels
url = 'https://github.com/ultralytics/assets/releases/download/v0.0.0/'
urls = [url + ('coco2017labels-segments.zip' if segments else 'coco2017labels.zip')]  # labels
download(urls, dir=dataset_dir / 'labels')

# Download images
urls = [
    'http://images.cocodataset.org/zips/train2017.zip',  # 19G, 118k images
    'http://images.cocodataset.org/zips/val2017.zip'  # 1G, 5k images
    # ,'http://images.cocodataset.org/zips/test2017.zip'  # 7G, 41k images (optional)
]
download(urls, dir=dataset_dir / 'images', threads=3)

print(f"Labels saved to {dataset_dir / 'labels'}")
print(f"Images saved to {dataset_dir / 'images'}")


# Image reordering
Copies all the images from ~/datasets/coco into the /dataset folder

In [ ]:
%reset -f
import os
import shutil
from tqdm import tqdm

# Define source directories
coco_train_images = os.path.expanduser("datasets/images/train2017")
coco_val_images = os.path.expanduser("datasets/images/val2017")
coco_train_labels = os.path.expanduser("datasets/labels/coco/labels/train2017")
coco_val_labels = os.path.expanduser("datasets/labels/coco/labels/val2017")

# Define target directories
dataset_images = "datasets/images"
dataset_labels = "datasets/labels"

# Ensure target directories exist
os.makedirs(dataset_images, exist_ok=True)
os.makedirs(dataset_labels, exist_ok=True)

# Helper function to copy files
def copy_files(source_dir, target_dir, file_extension, phase_name):
    """
    Copy files with a specific extension from source_dir to target_dir.

    Args:
        source_dir (str): Source directory path.
        target_dir (str): Target directory path.
        file_extension (str): File extension to filter.
        phase_name (str): Name of the phase (e.g., 'train', 'val') for progress.
    """
    files = [f for f in os.listdir(source_dir) if f.endswith(file_extension)]
    
    with tqdm(total=len(files), desc=f"Copying {phase_name} files", unit="file") as pbar:
        for file_name in files:
            src_path = os.path.relpath(os.path.join(source_dir, file_name))
            dst_path = os.path.relpath(os.path.join(target_dir, file_name))
            shutil.move(src_path, dst_path)
            pbar.update(1)

# Copy files
copy_files(coco_train_images, dataset_images, ".jpg", "Training Images")
copy_files(coco_val_images, dataset_images, ".jpg", "Validation Images")
copy_files(coco_train_labels, dataset_labels, ".txt", "Training Labels")
copy_files(coco_val_labels, dataset_labels, ".txt", "Validation Labels")

print("Files successfully copied to dataset/images and dataset/labels with relative paths.")




Clear unused files that have been downloaded from the COCO dataset

In [ ]:
import os

# Directories to clean
directories = [
    "datasets/images",
    "datasets/labels"
]

def clean_directory(directory, extensions=[".jpg", ".txt"]):
    """
    Removes all files and folders from a directory except those with the specified extensions.
    
    Args:
        directory (str): The directory to clean.
        extensions (list): The file extensions to keep (default: [".jpg", ".txt"]).
    """
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        # Check if the item is a file and doesn't have a desired extension
        if os.path.isfile(item_path) and not any(item.lower().endswith(ext) for ext in extensions):
            os.remove(item_path)  # Remove the file
        # If the item is a folder, remove it
        elif os.path.isdir(item_path):
            for root, dirs, files in os.walk(item_path, topdown=False):
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    os.rmdir(os.path.join(root, name))
            os.rmdir(item_path)  # Remove the directory itself

# Clean each directory
for directory in directories:
    if os.path.exists(directory):
        clean_directory(directory)
        print(f"Cleaned directory: {directory}")
    else:
        print(f"Directory does not exist: {directory}")

print("Cleaning complete!")


### Reformat dataset to include only vehicles.
The original dataset contains the labels of:

    ["aeroplane", "bicyclebike", "bird", "boat", "bottle", "bus",
    "car", "cat", "chair", "cow", "diningtable", "dog", "horse",
    "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

A new filtered label directory has been created so that only remain the vehicles:

    ["car", "bus", "motorbike", "bicyclebike"]

These new labels are stored in dataset/labels_filtered

In [ ]:
import os
from tqdm import tqdm

# Directory containing YOLO label .txt files
label_dir = "datasets/labels"  # Replace with your label directory path

# Mapping of original class IDs to new class IDs
CLASS_MAPPING = {0: 0, 1: 1, 2: 2, 3: 3, 5: 4, 6: 5, 7: 6}

# Directory to save filtered labels
output_dir = "datasets/labels_filtered"
os.makedirs(output_dir, exist_ok=True)

def filter_and_map_labels(label_file):
    """
    Reads a YOLO label file, filters out unwanted classes,
    maps allowed classes to new values, and writes the updated labels to a new file.
    """
    input_path = os.path.join(label_dir, label_file)
    output_path = os.path.join(output_dir, label_file)

    with open(input_path, "r") as infile, open(output_path, "w") as outfile:
        for line in infile:
            parts = line.split()
            class_id = int(parts[0])  # Extract class ID

            if class_id in CLASS_MAPPING:
                new_class_id = CLASS_MAPPING[class_id]
                # Replace class ID and write the updated line
                outfile.write(f"{new_class_id} " + " ".join(parts[1:]) + "\n")

# List all .txt files in the label directory
label_files = [f for f in os.listdir(label_dir) if f.endswith(".txt")]

# Process all .txt files with a progress bar
with tqdm(total=len(label_files), desc="Filtering & Mapping Labels", unit="file") as pbar:
    for file_name in label_files:
        filter_and_map_labels(file_name)
        pbar.update(1)

print(f"Filtered and mapped labels saved in: {output_dir}")


### Remove empty labels

In [ ]:
# Define the path to the labels folder
labels_folder = "datasets/labels_filtered"

# List all .txt files in the labels folder
label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]

# Initialize a counter for removed files
removed_count = 0

# Check each label file and remove it if it's empty
for label_file in label_files:
    label_path = os.path.join(labels_folder, label_file)
    if os.path.getsize(label_path) == 0:  # Check if the file size is 0 bytes
        os.remove(label_path)  # Remove the empty file
        removed_count += 1
        # print(f"Removed empty label: {label_file}")

# Output the result
print(f"Total empty labels removed: {removed_count}")



Counting images and filtered_labels

In [ ]:
print(os.getcwd())
# Define the paths to the images and labels folders
images_folder = "datasets/images"
labels_folder = "datasets/labels_filtered"

# List all files in the images and labels folders
image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]
label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]

# Count the total number of images and labels
num_images = len(image_files)
num_labels = len(label_files)

# Check for matching files (base filenames without extensions)
image_basenames = {os.path.splitext(f)[0] for f in image_files}
label_basenames = {os.path.splitext(f)[0] for f in label_files}

# Count matched and unmatched files
matched_files = image_basenames & label_basenames
unmatched_images = image_basenames - label_basenames
unmatched_labels = label_basenames - image_basenames

print(f"Total images: {num_images}")
print(f"Total labels: {num_labels}")
print(f"Matched files: {len(matched_files)}")
print(f"Unmatched images: {len(unmatched_images)}")
print(f"Unmatched labels: {len(unmatched_labels)}")

# Optionally print the unmatched files
# if unmatched_images:
#     print("Unmatched images (no corresponding label):")
#     for img in unmatched_images:
#         # print(f"  {img}")

# if unmatched_labels:
#     print("Unmatched labels (no corresponding image):")
#     for lbl in unmatched_labels:
#         # print(f"  {lbl}")

Create the folder of images_filtered with a reduced number of unlabellel images.
The ratio of labelled images and unlabelled images has been set to 50/50. 

In [ ]:
import random

# Paths
images_folder = "datasets/images"
labels_folder = "datasets/labels_filtered"
output_folder = "datasets/images_filtered"

# Ratio of labeled and unlabeled images
r_label = 50
r_unlabel = 100 - r_label

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Get all image files and corresponding label files
image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]
label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]

# Get the base filenames (without extensions) for labels
label_basenames = {os.path.splitext(label)[0] for label in label_files}

# Separate labeled and unlabeled images
labeled_images = [img for img in image_files if os.path.splitext(img)[0] in label_basenames]
unlabeled_images = [img for img in image_files if os.path.splitext(img)[0] not in label_basenames]

# Check counts
num_labeled = len(labeled_images)
num_unlabeled_to_select = min(int(num_labeled * r_unlabel / r_label), len(unlabeled_images))

# Randomly select the required number of unlabeled images
selected_unlabeled_images = random.sample(unlabeled_images, num_unlabeled_to_select)

# Combine labeled and selected unlabeled images
images_to_copy = labeled_images + selected_unlabeled_images

# Copy labeled and selected unlabeled images to the output folder with a progress bar
with tqdm(total=len(images_to_copy), desc="Copying Images", unit="file") as pbar:
    for img in images_to_copy:
        src_path = os.path.join(images_folder, img)
        dst_path = os.path.join(output_folder, img)
        shutil.copy(src_path, dst_path)
        pbar.update(1)

# Output results
print(f"Total labeled images: {num_labeled}")
print(f"Total unlabeled images selected: {len(selected_unlabeled_images)}")
print(f"Total images in 'images_filtered': {len(os.listdir(output_folder))}")



In [5]:
import os
import shutil
from collections import defaultdict
from tqdm import tqdm

# Paths
images_folder = "datasets/images"
labels_folder = "datasets/labels_filtered"

# Step 1: Count class occurrences across all label files
class_counts = defaultdict(int)
file_classes = {}  # Store classes per file

# Get label files
label_files = [f for f in os.listdir(labels_folder) if f.endswith(".txt")]

for label_file in tqdm(label_files, desc="Counting Class Frequencies", unit="file"):
    label_path = os.path.join(labels_folder, label_file)
    with open(label_path, "r") as f:
        lines = f.readlines()
    
    # Store class IDs present in this file
    file_classes[label_file] = set()
    
    for line in lines:
        class_id = int(line.split()[0])  # Extract class ID
        class_counts[class_id] += 1
        file_classes[label_file].add(class_id)

# Step 2: Identify dominant class and second most common class
sorted_classes = sorted(class_counts.items(), key=lambda x: x[1], reverse=True)
most_common_class = sorted_classes[0][0]  # Most frequent class
second_most_common_class_count = sorted_classes[1][1]  # Count of second most frequent class

# Define threshold for most common class
threshold = 2 * second_most_common_class_count
print(f"\nMost common class: {most_common_class} (Threshold: {threshold})")

# Step 3: Remove images where only the most common class exists
removed_files = 0
for label_file, classes in tqdm(file_classes.items(), desc="Filtering Dominant Class Images", unit="file"):
    if most_common_class in classes and len(classes) == 1:  # Only contains the dominant class
        # Remove corresponding image and label
        label_path = os.path.join(labels_folder, label_file)
        image_path = os.path.join(images_folder, label_file.replace(".txt", ".jpg"))  # Assuming .jpg images
        
        os.remove(label_path)
        if os.path.exists(image_path):
            os.remove(image_path)
        
        removed_files += 1
        class_counts[most_common_class] -= len(classes)  # Reduce count

        # Stop removing once threshold is met
        if class_counts[most_common_class] <= threshold:
            break

print(f"\nRemoved {removed_files} images where only class {most_common_class} was present.")
print(f"Final count of class {most_common_class}: {class_counts[most_common_class]}")

Counting Class Frequencies: 100%|██████████| 74784/74784 [08:10<00:00, 152.44file/s]



Most common class: 0 (Threshold: 90898)


Filtering Dominant Class Images: 100%|██████████| 74784/74784 [00:27<00:00, 2735.21file/s]


Removed 51048 images where only class 0 was present.
Final count of class 0: 216981


# Create Train, Validation and Test image sets
From the image and labels ("dataset/images", "dataset/labels_filtered")
Create the test, validation and test sets.


- Training is stored in ("dataset/train/images", "dataset/train/labels")
- Validation is stored in ("dataset/valid/images", "dataset/valid/labels")
- Test is stored in ("dataset/test/images", "dataset/test/labels")


The following code takes the unfiltered images and create the corresponding training, validation and test set in the following folders:
- dataset/train
- dataset/valid
- dataset/test

In [ ]:
# import os
# import shutil
# import random
# from tqdm import tqdm

# # Split data into 20% train, 5% validation, and 5% test
# train_perc = 0.8
# valid_perc = 0.1
# test_perc = 0.1

# # Define folder paths
# images_folder = "datasets/images"  # Folder containing filtered images
# labels_folder = "datasets/labels"  # Folder containing filtered labels

# train_images_folder = "datasets/train/images"
# train_labels_folder = "datasets/train/labels"

# valid_images_folder = "datasets/valid/images"
# valid_labels_folder = "datasets/valid/labels"

# test_images_folder = "datasets/test/images"
# test_labels_folder = "datasets/test/labels"

# # Create output directories
# for folder in [train_images_folder, train_labels_folder,
#                valid_images_folder, valid_labels_folder,
#                test_images_folder, test_labels_folder]:
#     if os.path.exists(folder):
#         # Remove the folder and its contents
#         shutil.rmtree(folder, ignore_errors=True)
    
#     os.makedirs(folder, exist_ok=True)

# # Get a list of all images
# image_files = sorted(os.listdir(images_folder))

# # Create a list of images with and without labels
# data = []
# for image_file in image_files:
#     label_file = os.path.splitext(image_file)[0] + ".txt"
#     if os.path.exists(os.path.join(labels_folder, label_file)):
#         data.append((image_file, label_file))  # Image has a corresponding label
#     else:
#         data.append((image_file, None))  # Image has no label (no objects detected)

# # Shuffle the data
# random.shuffle(data)

# # Calculate splits
# total_data = len(data)
# train_split = int(train_perc * total_data)
# valid_split = train_split + int(valid_perc * total_data)
# test_split = valid_split + int(test_perc * total_data)

# # Allocate data
# train_data = data[:train_split]
# valid_data = data[train_split:valid_split]
# test_data = data[valid_split:test_split]

# # Function to copy images and labels with a progress bar
# def copy_files(data, dest_images_folder, dest_labels_folder, phase_name):
#     with tqdm(total=len(data), desc=f"Copying {phase_name}") as pbar:
#         for image_file, label_file in data:
#             # Copy the image file
#             shutil.copy(os.path.join(images_folder, image_file), os.path.join(dest_images_folder, image_file))
#             # Copy the label file if it exists
#             if label_file:
#                 shutil.copy(os.path.join(labels_folder, label_file), os.path.join(dest_labels_folder, label_file))
#             # Update progress bar
#             pbar.update(1)

# # Copy data to respective folders
# copy_files(train_data, train_images_folder, train_labels_folder, "Training Data")
# copy_files(valid_data, valid_images_folder, valid_labels_folder, "Validation Data")
# copy_files(test_data, test_images_folder, test_labels_folder, "Testing Data")

# print("Dataset split complete!")
# print(f"Training data: {len(train_data)} images")
# print(f"Validation data: {len(valid_data)} images")
# print(f"Testing data: {len(test_data)} images")


The following code takes the filtered images and create the corresponding training, validation and test set in the following folders:
- dataset/train_filtered
- dataset/valid_filtered
- dataset/test_filtered


In [ ]:
import os
import shutil
import random
from tqdm import tqdm

# Split data into 20% train, 5% validation, and 5% test
train_perc = 0.8
valid_perc = 0.1
test_perc = 0.1

# Define folder paths
images_folder = "datasets/images_filtered"  # Folder containing filtered images
labels_folder = "datasets/labels_filtered"  # Folder containing filtered labels

train_images_folder = "datasets/train_filtered/images"
train_labels_folder = "datasets/train_filtered/labels"

valid_images_folder = "datasets/valid_filtered/images"
valid_labels_folder = "datasets/valid_filtered/labels"

test_images_folder = "datasets/test_filtered/images"
test_labels_folder = "datasets/test_filtered/labels"

# Create output directories
for folder in [train_images_folder, train_labels_folder,
               valid_images_folder, valid_labels_folder,
               test_images_folder, test_labels_folder]:
    if os.path.exists(folder):
        # Remove the folder and its contents
        shutil.rmtree(folder, ignore_errors=True)
    
    os.makedirs(folder, exist_ok=True)

# Get a list of all images
image_files = sorted(os.listdir(images_folder))

# Create a list of images with and without labels
data = []
for image_file in image_files:
    label_file = os.path.splitext(image_file)[0] + ".txt"
    if os.path.exists(os.path.join(labels_folder, label_file)):
        data.append((image_file, label_file))  # Image has a corresponding label
    else:
        data.append((image_file, None))  # Image has no label (no objects detected)

# Shuffle the data
random.shuffle(data)

# Calculate splits
total_data = len(data)
train_split = int(train_perc * total_data)
valid_split = train_split + int(valid_perc * total_data)
test_split = valid_split + int(test_perc * total_data)

# Allocate data
train_data = data[:train_split]
valid_data = data[train_split:valid_split]
test_data = data[valid_split:test_split]

# Function to copy images and labels with a progress bar
def copy_files(data, dest_images_folder, dest_labels_folder, phase_name):
    with tqdm(total=len(data), desc=f"Copying {phase_name}") as pbar:
        for image_file, label_file in data:
            # Copy the image file
            shutil.copy(os.path.join(images_folder, image_file), os.path.join(dest_images_folder, image_file))
            # Copy the label file if it exists
            if label_file:
                shutil.copy(os.path.join(labels_folder, label_file), os.path.join(dest_labels_folder, label_file))
            # Update progress bar
            pbar.update(1)

# Copy data to respective folders
copy_files(train_data, train_images_folder, train_labels_folder, "Training Data")
copy_files(valid_data, valid_images_folder, valid_labels_folder, "Validation Data")
copy_files(test_data, test_images_folder, test_labels_folder, "Testing Data")

print("Dataset split complete!")
print(f"Training data: {len(train_data)} images")
print(f"Validation data: {len(valid_data)} images")
print(f"Testing data: {len(test_data)} images")


In [3]:
import os
from collections import defaultdict
from tqdm import tqdm

# Directories to process
directories = [
    "datasets/labels",
    "datasets/train/labels",
    "datasets/valid/labels",
    "datasets/test/labels",
    
    "datasets/labels_filtered",
    "datasets/train_filtered/labels",
    "datasets/valid_filtered/labels",
    "datasets/test_filtered/labels",
]

def count_labels_in_directory(directory):
    """
    Counts the number of each class in a directory of YOLO label files with a progress bar.
    
    Args:
        directory (str): Path to the directory containing label files.
        
    Returns:
        dict: A dictionary with class IDs as keys and counts as values.
    """
    class_counts = defaultdict(int)
    
    # Get all .txt files in the directory
    label_files = [f for f in os.listdir(directory) if f.endswith(".txt")]
    
    # Process each file with a progress bar
    with tqdm(total=len(label_files), desc=f"Processing {directory}", unit="file") as pbar:
        for label_file in label_files:
            label_path = os.path.join(directory, label_file)
            with open(label_path, "r") as file:
                for line in file:
                    parts = line.split()
                    class_id = int(parts[0])  # Extract class ID
                    class_counts[class_id] += 1  # Increment the count for the class ID
            pbar.update(1)
    
    return class_counts

# Count labels for each directory
results = {}

for directory in directories:
    if os.path.exists(directory):
        results[directory] = count_labels_in_directory(directory)
    else:
        results[directory] = None  # Directory does not exist

# Display the results
print("\nClass counts per directory:")
for directory, class_counts in results.items():
    print(f"\nDirectory: {directory}")
    if class_counts is not None:
        for class_id, count in sorted(class_counts.items()):
            print(f"  Class {class_id}: {count}")
    else:
        print("  Directory does not exist or contains no labels.")



Processing datasets/test_filtered/labels: 100%|██████████| 7473/7473 [00:43<00:00, 170.27file/s]


Class counts per directory:

Directory: datasets/labels
  Class 0: 268029
  Class 1: 7370
  Class 2: 45449
  Class 3: 9021
  Class 4: 5272
  Class 5: 6344
  Class 6: 4760
  Class 7: 10384
  Class 8: 11000
  Class 9: 13476
  Class 10: 1966
  Class 11: 2058
  Class 12: 1343
  Class 13: 10231
  Class 14: 10969
  Class 15: 4968
  Class 16: 5718
  Class 17: 6839
  Class 18: 9577
  Class 19: 8386
  Class 20: 5736
  Class 21: 1365
  Class 22: 5535
  Class 23: 5360
  Class 24: 9084
  Class 25: 11672
  Class 26: 12882
  Class 27: 6700
  Class 28: 6411
  Class 29: 2796
  Class 30: 6864
  Class 31: 2750
  Class 32: 6559
  Class 33: 9129
  Class 34: 3418
  Class 35: 3895
  Class 36: 5715
  Class 37: 6362
  Class 38: 5032
  Class 39: 25083
  Class 40: 8180
  Class 41: 21469
  Class 42: 5689
  Class 43: 8085
  Class 44: 6412
  Class 45: 14946
  Class 46: 9565
  Class 47: 6012
  Class 48: 4533
  Class 49: 6587
  Class 50: 7573
  Class 51: 8123
  Class 52: 3008
  Class 53: 6091
  Class 54: 7333
  Cla

# Load YOLO Model and Begin Training!!

In [ ]:
import torch

# Check for available devices
device_name = "cuda" if torch.cuda.is_available() else "cpu"
available_devices = torch.cuda.device_count()

print(f"Device in use: {device_name}")
print(f"Available CUDA devices: {available_devices}")
from ultralytics import YOLO
!yolo checks

### Hyperparam tuning

In [ ]:
# from ultralytics import YOLO
# model = YOLO("yolo11n.yaml")
# model.tune(
#     data="train_filtered.yaml",
#     project="./hyperparam_tuning3",
#     pretrained=False,  
#     epochs = 5,
#     iterations=100,
#     )

### Training with base hyperparameters

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n.yaml")
model.train(
    data="train_filtered.yaml",
    project="models/base_16",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    batch=16,
    )

In [ ]:
from ultralytics import YOLO

model32 = YOLO("yolo11n.yaml")
model32.train(
    data="train_filtered.yaml",
    project="models/base_32",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    batch=32,
    )

In [ ]:
from ultralytics import YOLO

modelX = YOLO("yolo11n.yaml")
modelX.train(
    data="train_filtered.yaml",
    project="models/base_X",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    batch=-1,
    )

### Training with tuned 1 hyperparameters

In [ ]:
from ultralytics import YOLO

model16 = YOLO("yolo11n.yaml")
model16.train(
    data="train_filtered.yaml",
    cfg="tunings/tuning1/tune/best_hyperparameters.yaml",
    project="models/tuned1_16",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    batch=16,
    )

In [ ]:
from ultralytics import YOLO

model32 = YOLO("yolo11n.yaml")
model32.train(
    data="train_filtered.yaml",
    cfg="tunings/tuning1/tune/best_hyperparameters.yaml",
    project="models/tuned1_32",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    batch=32,
    )

In [4]:
from ultralytics import YOLO

modelX = YOLO("yolo11n.yaml")
modelX.train(
    data="train_filtered.yaml",
    cfg="tunings/tuning1/tune/best_hyperparameters.yaml",
    project="models/tuned1_X",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    batch=-1,
    )

New https://pypi.org/project/ultralytics/8.3.82 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.78  Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.yaml, data=train_filtered.yaml, epochs=200, time=None, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=models/tuned1_X, name=train, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=N

train: Scanning C:\Users\Usuario\Documents\TFM\ForeHelm\training\COCO\datasets\train_filtered\labels.cache... 59806 images, 38823 backgrounds, 0 corrupt: 100%|██████████| 98629/98629 [00:00<?, ?it/s]


AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU) 8.00G total, 0.09G reserved, 0.05G allocated, 7.86G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     2591205       6.447         0.440         73.64         478.2        (1, 3, 640, 640)                    list
     2591205       12.89         0.596         76.09         138.9        (2, 3, 640, 640)                    list
     2591205       25.79         0.929         65.01           161        (4, 3, 640, 640)                    list
     2591205       51.58         1.569         45.61         112.3        (8, 3, 640, 640)                    list
     2591205       103.2         2.840         67.24         149.1       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 39 for CUDA:0 4.89G/8.00G (61%) 


train: Scanning C:\Users\Usuario\Documents\TFM\ForeHelm\training\COCO\datasets\train_filtered\labels.cache... 59806 images, 38823 backgrounds, 0 corrupt: 100%|██████████| 98629/98629 [00:00<?, ?it/s]
val: Scanning C:\Users\Usuario\Documents\TFM\ForeHelm\training\COCO\datasets\valid_filtered\labels... 7504 images, 4824 backgrounds, 0 corrupt: 100%|██████████| 12328/12328 [00:13<00:00, 929.90it/s] 


val: New cache created: C:\Users\Usuario\Documents\TFM\ForeHelm\training\COCO\datasets\valid_filtered\labels.cache
Plotting labels to models\tuned1_X\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.00938' and 'momentum=0.89665' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005971875), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to models\tuned1_X\train
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200      6.33G      3.745      8.099      5.149        211        640:  77%|███████▋  | 1946/2529 [11:23<03:24,  2.85it/s]


KeyboardInterrupt: 

### Training with tuned 2 hyperparameters

In [ ]:
from ultralytics import YOLO

model16 = YOLO("yolo11n.yaml")
model16.train(
    data="train_filtered.yaml",
    cfg="tunings/tuning2/tune/best_hyperparameters.yaml",
    project="models/tuned2_16",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    batch=16,
    )

In [ ]:
from ultralytics import YOLO

model32 = YOLO("yolo11n.yaml")
model32.train(
    data="train_filtered.yaml",
    cfg="tunings/tuning2/tune/best_hyperparameters.yaml",
    project="models/tuned2_32",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    batch=32,
    )

In [ ]:
from ultralytics import YOLO

modelX = YOLO("yolo11n.yaml")
modelX.train(
    data="train_filtered.yaml",
    cfg="tunings/tuning2/tune/best_hyperparameters.yaml",
    project="models/tuned2_X",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    batch=-1,
    )

In [ ]:
from ultralytics import YOLO
model16 = YOLO("models/base_16/train/weights/last.pt")
model16.train(resume=True)

# Validating the YOLO models
The following code is used to validate and compare the different YOLO models that have been trained

This code tries the default and pretrained yolo11n model with the COCO dataset

In [ ]:
from ultralytics import YOLO

# List of model weight paths
weights_paths = [
    "yolo11n.pt",
    "models/base_16/train/weights/best.pt",
    "models/base_32/train/weights/best.pt",
    "models/base_X/train/weights/best.pt",
    "models/tuned1_16/train/weights/best.pt",
    "models/tuned1_32/train2/weights/best.pt",
    "models/tuned21_X/train/weights/best.pt",
    "models/tuned2_16/train/weights/best.pt",
    "models/tuned2_32/train/weights/best.pt",
    "models/tuned2_X/train/weights/best.pt"
]

# Corresponding project names for each model
prj_paths = [
    "yolo11n",
    "models/base_16",
    "models/base_32",
    "models/base_X",
    "models/tuned1_16",
    "models/tuned1_32",
    "models/tuned21_X",
    "models/tuned2_16",
    "models/tuned2_32",
    "models/tuned2_X"
]

In [ ]:
# Path to your dataset configuration file
data_path = "train_filtered.yaml"
# Device to run the validation on (e.g., '0' for the first GPU, 'cpu' for CPU)
device = "0"
# Dictionary to store validation results
validation_results = {}

# Iterate over each model weight path and its corresponding project name
for weights_path, prj_path in zip(weights_paths, prj_paths):
    # Load the model
    model = YOLO(weights_path)    
    # Perform validation
    results = model.val(data=data_path, project=prj_path, device=device, save_json=True, save=False)
    # Extract relevant metrics
    metrics = {
        'mAP50': results.box.map50,
        'mAP50-95': results.box.map,
        'mAP75': results.box.map75,
        'mAPs': results.box.maps
    }
    # Store metrics in the dictionary
    validation_results[prj_path] = metrics

In [ ]:
print(validation_results)

In [ ]:
from ultralytics.utils.benchmarks import benchmark
benchmark_results = {}
for weights_path, prj_path in zip(weights_paths, prj_paths):
    # Benchmark specific export format
    benchmark_results[prj_path] = benchmark(model=weights_path, data=data_path, imgsz=640, format="ncnn")

In [ ]:
import os
from ultralytics import YOLO

for weights_path in weights_paths:
    # Load the model
    model = YOLO(weights_path)
    # Export the model to NCNN format
    model.export(format="ncnn")


In [ ]:
# Path to your dataset configuration file
data_path = "train_filtered.yaml"
# Device to run the validation on (e.g., '0' for the first GPU, 'cpu' for CPU)
device = "0"
# Dictionary to store validation results
validation_results_ncnn = {}

# Iterate over each model weight path and its corresponding project name
for weights_path, prj_path in zip(weights_ncnn_paths, prj_paths):
    # Load the model
    model = YOLO(weights_path)
    
    # Perform validation
    results = model.val(data=data_path, project=prj_path, name="val_ncnn",device=device, save_json=True, save=False)
    
    # Extract relevant metrics
    metrics = {
        'mAP50': results.box.map50,
        'mAP50-95': results.box.map,
        'mAP75': results.box.map75,
        'mAPs': results.box.maps
    }
    
    # Store metrics in the dictionary
    validation_results_ncnn[weights_path] = metrics

## Model Prunning

In [ ]:
import torch
import torch.nn.utils.prune as prune
from ultralytics import YOLO
import numpy as np
def prune_model_l1(model, amount=0.2):
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')
    return model

def prune_model_l2(model, amount=0.2):
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.l2_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')
    return model

def prune_model_global(model, amount=0.2):
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.global_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')
    return model

In [ ]:
# Load the YOLO model
weights = "models/tuned1_16/train/weights/best.pt"
model = YOLO(weights)

# Validate the original model (optional)
# result = model.val(data="train_filtered.yaml")
print("Original model validation completed.")
torch_model = model.model
print(torch_model)

results_l1 = {}
results_l2 = {}
results_global = {}

for i in np.arange(0.01, 0.21, 0.02):
    # Prune the model by updating the model's internal torch model in place
    print("Pruning model...")
    pruned_model = prune_model_l1(torch_model, amount=i)
    print(f"Model pruned for {i}.")
    model.model = pruned_model
    # Validate the pruned model using the same YOLO validation method
    result = model.val(data="train_filtered.yaml")
    results_l1[str(i)] =  result
    print(f"Pruned model validation completed for {i}.")


for i in np.arange(0.01, 0.21, 0.02):
    # Prune the model by updating the model's internal torch model in place
    print("Pruning model...")
    pruned_model = prune_model_l2(torch_model, amount=i)
    print(f"Model pruned for {i}.")
    model.model = pruned_model
    # Validate the pruned model using the same YOLO validation method
    result = model.val(data="train_filtered.yaml")
    results_l2[str(i)] =  result
    print(f"Pruned model validation completed for {i}.")


for i in np.arange(0.01, 0.21, 0.02):
    # Prune the model by updating the model's internal torch model in place
    print("Pruning model...")
    pruned_model = prune_model_global(torch_model, amount=i)
    print(f"Model pruned for {i}.")
    model.model = pruned_model
    # Validate the pruned model using the same YOLO validation method
    result = model.val(data="train_filtered.yaml")
    results_global[str(i)] =  result
    print(f"Pruned model validation completed for {i}.")


In [ ]:
print(results_l1)


In [ ]:
print(results_l2)


In [ ]:
print(results_global)

In [ ]:
model = YOLO("base_40/train/weights/best.pt")
results = model.val(data="train_filtered.yaml")